# Hugging Face Transformers 微调训练入门

In [1]:
from datasets import Dataset, DatasetDict

In [2]:
# Yelp评论完整星级数据集是通过随机选取每个1到5星评论的130,000个训练样本和10,000个测试样本构建的。总共有650,000个训练样本和50,000个测试样本。
train_full_ds = Dataset.from_parquet("r_data/yelp_review_full/train-00000-of-00001.parquet")
test_full_ds = Dataset.from_parquet("r_data/yelp_review_full/test-00000-of-00001.parquet")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
dataset = DatasetDict({
    'train': train_full_ds,
    'test': test_full_ds
})

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [5]:
dataset["train"][111]

{'label': 2,
 'text': "As far as Starbucks go, this is a pretty nice one.  The baristas are friendly and while I was here, a lot of regulars must have come in, because they bantered away with almost everyone.  The bathroom was clean and well maintained and the trash wasn't overflowing in the canisters around the store.  The pastries looked fresh, but I didn't partake.  The noise level was also at a nice working level - not too loud, music just barely audible.\\n\\nI do wish there was more seating.  It is nice that this location has a counter at the end of the bar for sole workers, but it doesn't replace more tables.  I'm sure this isn't as much of a problem in the summer when there's the space outside.\\n\\nThere was a treat receipt promo going on, but the barista didn't tell me about it, which I found odd.  Usually when they have promos like that going on, they ask everyone if they want their receipt to come back later in the day to claim whatever the offer is.  Today it was one of th

In [6]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [7]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(dataset["train"])

,label,text
0,5 stars,"My husband and I first tried this food truck while at Urban Cookies, and now .... we log on at least weekly to see where we can follow them to. Ya, it can be a little pricey, but soooo worth it. And the luncharitas are amazing!"
1,4 stars,"I discovered this place while waiting for the 64 bus after leaving the Manor movie theatre. The place was small and the sign in front is barely noticeable and I fully expected to see Dozens bakery when I looked through the window, but I was pleasantly surprised to see waffle irons at work. I have been dying to find a place that serves fresh hot waffles for a long time now. My boyfriend and I tried the fancified versions of the waffle at first, mine topped with whipped cream and fresh strawberries and his with a chocolate-hazelnut spread (I can't remember if it was Nutella or some other brand) and sliced bananas. Regardless of the small \""dining\"" area we were fully satisfied with our waffles. They were warm and fluffy and light so you get a fulfilling snack without it being too heavy. Perfect snack after the movie while we waited for the bus. We stopped in on a different occasion after dinner on a cold rainy day to grab a few of the plain sugar waffles to go. These were perfectly portable and delicious. I think I prefer them this way, without all of the extra toppings. \n\nOne other thing that I like about this place is that they accept cards, which is very convenient for people like me, who never carry any cash. \n\nIt's too bad that I don't live in Squirrel Hill anymore. Otherwise, I'd stop by more often. They need to advertise their shop a bit more because I had no idea that they were there. I even missed the sign in front. I love the idea of the waffles being made right in front of the glass window though."
2,5 stars,"I'm still angry at my best buddy, who lives/lived in Vegas, for not taking me here sooner. HOLY COW. SO DELICIOUS. I cry when I think of the red velvet pancakes. I cry even more when thinking of the kalua pig omelette. \n\nWe went for brunch on Friday at around 11:30 AM. There were a group of 6 before us but other than that, not much of a wait. We got our seats within 10 minutes. Love that they had Hawaii-an food selection. i ordered the kalua pig omelette while one bff got a crepe and the other bff (who i'm mad at), got the red velvet panackes. \n\nThe crepe, was just a crepe. But the pancakes were warm and delicious and so much like a red velvet cupcake. A perfect accompaniment to my slightly salty kalua pig.\n\nThe kalua pig was perfectly salted and made me miss home just a bit. The portion was generous; if I were going home after brunch, I'd save some to eat for later. But I wasn't so I went ahead and ate most of it. My crepe-bff ate what I couldn't finish.\n\nI can't wait to come back to Vegas for this place! Slots/gambling/clubbing, I'm not such a huge fan. But this place, is worth flying back for."
3,3 stars,"Koi Las Vegas has some major hype going on around it! Kim Kardashian, Kevin Spacey, David Spade......if you flip on the Extra channel in the new Planet Hollywood Hotel, it seems like all the celebrities have something nice to say about Koi. The original Koi is in Los Angeles, and frequently gets celebrities hopping in and out of that restaurant. \n\nFood-wise, this place is no match for Matsuhisa in Los Angeles, but it is a nice place to have dinner if you're on the Strip. Koi Las Vegas is on the Mezzanine level in the Planet Hollywood Hotel & Casino. Three jolly buddhas are positioned in front of the restaurant, and then you walk around the circular lounge, and into the dimly lit restaurant flanked by modern grey/green booths and glossy brown tables. It's very dark in the restaurant, with just the sushi/sashimi chefs lit up under recessed lighting near the sushi bar. Lots of businessmen at this restaurant....those darn companies pick up the pricey tab when dinner is all over. Wish I could have a company expense! Some people were there f

## 预处理数据

下载数据集到本地后，使用 Tokenizer 来处理文本，对于长度不等的输入数据，可以使用填充（padding）和截断（truncation）策略来处理。

Datasets 的 `map` 方法，支持一次性在整个数据集上应用预处理函数。

下面使用填充到最大长度的策略，处理整个数据集：

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/data/models/bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [10]:
tokenized_datasets.save_to_disk("r_data/yelp_review_full/tokenized_datasets")

Saving the dataset (0/5 shards):   0%|          | 0/650000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/50000 [00:00<?, ? examples/s]

In [11]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [12]:
from datasets import load_from_disk
tokenized_datasets = load_from_disk("r_data/yelp_review_full/tokenized_datasets")

### 数据抽样

使用 N 个数据样本，在 BERT 上演示小规模训练（基于 Pytorch Trainer）

`shuffle()`函数会随机重新排列列的值。如果您希望对用于洗牌数据集的算法有更多控制，可以在此函数中指定generator参数来使用不同的numpy.random.Generator。

In [13]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=66).select(range(100))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=66).select(range(100))

## 微调训练配置

### 加载 BERT 模型

警告通知我们正在丢弃一些权重（`vocab_transform` 和 `vocab_layer_norm` 层），并随机初始化其他一些权重（`pre_classifier` 和 `classifier` 层）。在微调模型情况下是绝对正常的，因为我们正在删除用于预训练模型的掩码语言建模任务的头部，并用一个新的头部替换它，对于这个新头部，我们没有预训练的权重，所以库会警告我们在用它进行推理之前应该对这个模型进行微调，而这正是我们要做的事情。

In [14]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("/data/models/bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /data/models/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 训练超参数（TrainingArguments）

完整配置参数与默认值：
- https://huggingface.co/docs/transformers/v4.36.1/en/main_classes/trainer#transformers.TrainingArguments
- https://hf-mirror.com/docs/transformers/training

源代码定义：https://github.com/huggingface/transformers/blob/v4.36.1/src/transformers/training_args.py#L161

**最重要配置：模型权重保存路径(output_dir)**

In [15]:
from transformers import TrainingArguments

model_dir = "/data/models/bert-base-cased-finetune-yelp-100"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为50
training_args = TrainingArguments(output_dir=model_dir,
                                  per_device_train_batch_size=16,
                                  num_train_epochs=5,
                                  logging_steps=50,
                                  save_total_limit=5)

In [5]:
TrainingArguments?

Init signature:
TrainingArguments(
    output_dir: Optional[str] = None,
    overwrite_output_dir: bool = False,
    do_train: bool = False,
    do_eval: bool = False,
    do_predict: bool = False,
    eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no',
    prediction_loss_only: bool = False,
    per_device_train_batch_size: int = 8,
    per_device_eval_batch_size: int = 8,
    per_gpu_train_batch_size: Optional[int] = None,
    per_gpu_eval_batch_size: Optional[int] = None,
    gradient_accumulation_steps: int = 1,
    eval_accumulation_steps: Optional[int] = None,
    eval_delay: Optional[float] = 0,
    torch_empty_cache_steps: Optional[int] = None,
    learning_rate: float = 5e-05,
    weight_decay: float = 0.0,
    adam_beta1: float = 0.9,
    adam_beta2: float = 0.999,
    adam_epsilon: float = 1e-08,
    max_grad_norm: float = 1.0,
    num_train_epochs: float = 3.0,
    max_steps: int = -1,
    lr_scheduler_type: Union[transformers.trainer_utils.Schedu

In [6]:
# 完整的超参数配置
print(training_args)

TrainingArguments(
_n_gpu=0,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStrategy.NO,
eval_use_gather_object=False,


### 训练过程中的指标评估（Evaluate)

**[Hugging Face Evaluate 库](https://huggingface.co/docs/evaluate/index)** 支持使用一行代码，获得数十种不同领域（自然语言处理、计算机视觉、强化学习等）的评估方法。 当前支持 **完整评估指标：https://huggingface.co/evaluate-metric**

训练器（Trainer）在训练过程中不会自动评估模型性能。因此，我们需要向训练器传递一个函数来计算和报告指标。 

Evaluate库提供了一个简单的准确率函数，您可以使用`evaluate.load`函数加载

不能科学上网，可从 https://github.com/huggingface/evaluate 下载源码，然后在evaluate.load的时候，指定使用的metrics。

In [16]:
import numpy as np
import evaluate

In [19]:
metric = evaluate.load("/data/models/evaluate-0.4.5/metrics/accuracy")

接着，调用 `compute` 函数来计算预测的准确率。

在将预测传递给 compute 函数之前，我们需要将 logits 转换为预测值（**所有Transformers 模型都返回 logits**）。

In [20]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#### 训练过程指标监控

通常，为了监控训练过程中的评估指标变化，我们可以在`TrainingArguments`指定`evaluation_strategy`参数，以便在 epoch 结束时报告评估指标。

In [22]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=model_dir,
                                  #evaluation_strategy="epoch", 
                                  eval_strategy="epoch",
                                  per_device_train_batch_size=16,
                                  num_train_epochs=3,
                                  logging_steps=30)

## 开始训练

### 实例化训练器（Trainer）

需要升级accelerate
```bash
pip uninstall accelerate -y
pip cache purge
pip install accelerate --upgrade
```

In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [24]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.647452,0.230000
2,No log,1.660595,0.190000
3,No log,1.653814,0.210000


TrainOutput(global_step=21, training_loss=1.5462110610235305, metrics={'train_runtime': 2358.2756, 'train_samples_per_second': 0.127, 'train_steps_per_second': 0.009, 'total_flos': 78935442739200.0, 'train_loss': 1.5462110610235305, 'epoch': 3.0})

这是老师给的1000个样本的训练结果，可见样本增加，精度会提高。

TrainOutput(global_step=189, training_loss=0.9693943861300353, metrics={'train_runtime': 341.7098, 'train_samples_per_second': 8.779, 'train_steps_per_second': 0.553, 'total_flos': 789354427392000.0, 'train_loss': 0.9693943861300353, 'epoch': 3.0})

|Epoch|Training Loss|Validation Loss|Accuracy|
|-|-|-|-|
|1|1.242100|1.090886|0.526000|
|2|0.901400|0.960115|0.591000|
|3|0.638200|0.978361|0.592000|

In [25]:
small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(10))

In [26]:
trainer.evaluate(small_test_dataset)

{'eval_loss': 1.5807993412017822,
 'eval_accuracy': 0.3,
 'eval_runtime': 23.2638,
 'eval_samples_per_second': 0.43,
 'eval_steps_per_second': 0.086,
 'epoch': 3.0}

老师给的使用1000个样本训练，使用100个随机样本测试，结果如下
```
{'eval_loss': 1.0753791332244873,
 'eval_accuracy': 0.52,
 'eval_runtime': 2.9889,
 'eval_samples_per_second': 33.457,
 'eval_steps_per_second': 4.349,
 'epoch': 3.0}
```
 **说明样本量越大，训练的精度会同步提升。**

### 保存模型和训练状态

- 使用 `trainer.save_model` 方法保存模型，后续可以通过 from_pretrained() 方法重新加载
- 使用 `trainer.save_state` 方法保存训练状态

In [27]:
trainer.save_model(model_dir)

In [28]:
trainer.save_state()

# Hugging Face Transformers 微调语言模型-问答任务

我们已经学会使用 Pipeline 加载支持问答任务的预训练模型，本教程代码将展示如何微调训练一个支持问答任务的模型。

**注意：微调后的模型仍然是通过提取上下文的子串来回答问题的，而不是生成新的文本。**

In [1]:
# 根据你使用的模型和GPU资源情况，调整以下关键参数
squad_v2 = False
model_checkpoint = "/data/models/distilbert--distilbert-base-uncased"
batch_size = 16

## 下载数据集

在本教程中，我们将使用[斯坦福问答数据集(SQuAD）](https://rajpurkar.github.io/SQuAD-explorer/)。

https://hf-mirror.com/datasets/rajpurkar/squad_v2

### SQuAD 数据集

**斯坦福问答数据集(SQuAD)** 是一个阅读理解数据集，由众包工作者在一系列维基百科文章上提出问题组成。每个问题的答案都是相应阅读段落中的文本片段或范围，或者该问题可能无法回答。

SQuAD2.0将SQuAD1.1中的10万个问题与由众包工作者对抗性地撰写的5万多个无法回答的问题相结合，使其看起来与可回答的问题类似。要在SQuAD2.0上表现良好，系统不仅必须在可能时回答问题，还必须确定段落中没有支持任何答案，并放弃回答。

In [2]:
from datasets import Dataset, DatasetDict

In [3]:
squad_v2_train_full_ds = Dataset.from_parquet("r_data/squad_v2/train-00000-of-00001.parquet")
squad_v2_valid_full_ds = Dataset.from_parquet("r_data/squad_v2/validation-00000-of-00001.parquet")

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
datasets = DatasetDict({
    'train': squad_v2_train_full_ds,
    'validation': squad_v2_valid_full_ds
})

In [5]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [6]:
datasets["train"][0]

{'id': '56be85543aeaaa14008c9063',
 'title': 'Beyoncé',
 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'question': 'When did Beyonce start becoming popular?',
 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}}

In [7]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(datasets["train"])

,id,title,context,question,answers
0,5a84d5727cf838001a46aaec,Party_leaders_of_the_United_States_House_of_Representatives,"When the White House is controlled by the House majority party, then the House minority leader assumes a larger role in formulating alternatives to executive branch initiatives and in acting as a national spokesperson for his or her party. ""As Minority Leader during [President Lyndon Johnson's] Democratic administration, my responsibility has been to propose Republican alternatives,"" said Minority Leader Gerald Ford, R-MI. Greatly outnumbered in the House, Minority Leader Ford devised a political strategy that allowed Republicans to offer their alternatives in a manner that provided them political protection. As Ford explained:",What role does the executive branch assume when the house is controlled by the Republicans?,"{'text': [], 'answer_start': []}"
1,5726b375708984140094ce35,Pope_Paul_VI,"As a Cardinal, Montini journeyed to Africa (1962), where he visited Ghana, Sudan, Kenya, Congo, Rhodesia, South Africa, and Nigeria. After his journey, John XXIII gave him a private audience on his trip which lasted for several hours. In fifteen other trips he visited Brazil (1960) and the USA (1960), including New York City, Washington, DC, Chicago, the University of Notre Dame in Indiana, Boston, Philadelphia, and Baltimore. While a cardinal, he usually vacationed in Engelberg Abbey, a secluded Benedictine monastery in Switzerland.",Where did Montini like to use as a retreat as a cardinal?,"{'text': ['Engelberg Abbey'], 'answer_start': [474]}"
2,572719a45951b619008f85f7,Capacitor,"Capacitors may have their connecting leads arranged in many configurations, for example axially or radially. ""Axial"" means that the leads are on a common axis, typically the axis of the capacitor's cylindrical body – the leads extend from opposite ends. Radial leads might more accurately be referred to as tandem; they are rarely actually aligned along radii of the body's circle, so the term is inexact, although universal. The leads (until bent) are usually in planes parallel to that of the flat body of the capacitor, and extend in the same direction; they are often parallel as manufactured.",What is one type of configuration in which a capacitor may have its connecting leads organized?,"{'text': ['axially'], 'answer_start': [88]}"
3,5ad370ad604f3c001a3fe25e,Separation_of_powers_under_the_United_States_Constitution,"Separation of powers is a political doctrine originating in the writings of Montesquieu in The Spirit of the Laws where he urged for a constitutional government with three separate branches of government. Each of the three branches would have defined abilities to check the powers of the other branches. This idea was called separation of powers. This philosophy heavily influenced the writing of the United States Constitution, according to which the Legislative, Executive, and Judicial branches of the United States government are kept distinct in order to prevent abuse of power. This United States form of separation of powers is associated with a system of checks and balances.",Who urged for a government with four individual branches?,"{'text': [], 'answer_start': []}"
4,5731dbdf0fdd8d15006c65a6,Bras%C3%ADlia,"Brasília has a tropical savanna climate (Aw) according to the Köppen system, with two distinct seasons: the rainy season, from October to April, and a dry season, from May to September. The average temperature is 20.6 °C (69.1 °F). September, at the end of the dry season, has the highest average maximum temperature, 28.3 °C (82.9 °F), has major and minor lower maximum average temperature, of 25.1 °C (77.2 °F) and 12.9 °C (55.2 °F), respectively. Average temperatures from September through March are a consistent 22 °C (72 °F). With 247.4 mm (9.7 in), January is the month with the highest rainfall of the year, while June is the lowest, with only 8.7 mm (0.3 in).",When is Brasilia's rainy season?,"{'text': ['October to Apr

## 预处理数据

In [9]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

以下断言确保我们的 Tokenizers 使用的是 FastTokenizer（Rust 实现，速度和功能性上有一定优势）。

In [10]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

您可以在大模型表上查看哪种类型的模型具有可用的快速标记器，哪种类型没有。

您可以直接在两个句子上调用此标记器（一个用于答案，一个用于上下文）：

In [11]:
tokenizer("What is your name?", "My name is Sylvain.")

{'input_ids': [101, 2054, 2003, 2115, 2171, 1029, 102, 2026, 2171, 2003, 25353, 22144, 2378, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

### Tokenizer 进阶操作

在问答预处理中的一个特定问题是如何处理非常长的文档。

在其他任务中，当文档的长度超过模型最大句子长度时，我们通常会截断它们，但在这里，删除上下文的一部分可能会导致我们丢失正在寻找的答案。

为了解决这个问题，我们允许数据集中的一个（长）示例生成多个输入特征，每个特征的长度都小于模型的最大长度（或我们设置的超参数）。

In [12]:
# The maximum length of a feature (question and context)
max_length = 384 
# The authorized overlap between two part of the context when splitting it is needed.
doc_stride = 128 

#### 超出最大长度的文本数据处理

下面，我们从训练集中找出一个超过最大长度（384）的文本：

In [13]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
# 挑选出来超过384（最大长度）的数据样例
example = datasets["train"][i]

In [14]:
len(tokenizer(example["question"], example["context"])["input_ids"])

437

#### 截断上下文不保留超出部分

In [15]:
len(tokenizer(example["question"],
              example["context"],
              max_length=max_length,
              truncation="only_second")["input_ids"])

384

#### 关于截断的策略

- 直接截断超出部分: truncation=`only_second`
- 仅截断上下文（context），保留问题（question）：`return_overflowing_tokens=True` & 设置`stride`

In [16]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

使用此策略截断后，Tokenizer 将返回多个 `input_ids` 列表。

In [17]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 192]

解码两个输入特征，可以看到重叠的部分：

In [18]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[CLS] beyonce got married in 2008 to whom? [SEP] on april 4, 2008, beyonce married jay z. she publicly revealed their marriage in a video montage at the listening party for her third studio album, i am... sasha fierce, in manhattan ' s sony club on october 22, 2008. i am... sasha fierce was released on november 18, 2008 in the united states. the album formally introduces beyonce ' s alter ego sasha fierce, conceived during the making of her 2003 single " crazy in love ", selling 482, 000 copies in its first week, debuting atop the billboard 200, and giving beyonce her third consecutive number - one album in the us. the album featured the number - one song " single ladies ( put a ring on it ) " and the top - five songs " if i were a boy " and " halo ". achieving the accomplishment of becoming her longest - running hot 100 single in her career, " halo " ' s success in the us helped beyonce attain more top - ten singles on the list than any other woman during the 2000s. it also included t

#### 使用 offsets_mapping 获取原始的 input_ids

设置 `return_offsets_mapping=True`，将使得截断分割生成的多个 input_ids 列表中的 token，通过映射保留原始文本的 input_ids。

如下所示：第一个标记（[CLS]）的起始和结束字符都是（0, 0），因为它不对应问题/答案的任何部分，然后第二个标记与问题(question)的字符0到3相同.

In [19]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 7), (8, 11), (12, 19), (20, 22), (23, 27), (28, 30), (31, 35), (35, 36), (0, 0), (0, 2), (3, 8), (9, 10), (10, 11), (12, 16), (16, 17), (18, 25), (26, 33), (34, 37), (38, 39), (39, 40), (41, 44), (45, 53), (54, 62), (63, 68), (69, 77), (78, 80), (81, 82), (83, 88), (89, 93), (93, 96), (97, 99), (100, 103), (104, 113), (114, 119), (120, 123), (124, 127), (128, 133), (134, 140), (141, 146), (146, 147), (148, 149), (150, 152), (152, 153), (153, 154), (154, 155), (156, 161), (162, 168), (168, 169), (170, 172), (173, 182), (182, 183), (183, 184), (185, 189), (190, 194), (195, 197), (198, 205), (206, 208), (208, 209), (210, 214), (214, 215), (216, 217), (218, 220), (220, 221), (221, 222), (222, 223), (224, 229), (230, 236), (237, 240), (241, 249), (250, 252), (253, 261), (262, 264), (264, 265), (266, 270), (271, 273), (274, 277), (278, 284), (285, 291), (291, 292), (293, 296), (297, 302), (303, 311), (312, 322), (323, 330), (330, 331), (331, 332), (333, 338), (339, 342), (343, 3

因此，我们可以使用这个映射来找到答案在给定特征中的起始和结束标记的位置。

我们只需区分偏移的哪些部分对应于问题，哪些部分对应于上下文。

In [20]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

beyonce Beyonce


In [21]:
second_token_id = tokenized_example["input_ids"][0][2]
offsets = tokenized_example["offset_mapping"][0][2]
print(tokenizer.convert_ids_to_tokens([second_token_id])[0], example["question"][offsets[0]:offsets[1]])

got got


In [22]:
example["question"]

'Beyonce got married in 2008 to whom?'

借助`tokenized_example`的`sequence_ids`方法，我们可以方便的区分token的来源编号：

- 对于特殊标记：返回None，
- 对于正文Token：返回句子编号（从0开始编号）。

综上，现在我们可以很方便的在一个输入特征中找到答案的起始和结束 Token。

In [23]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [24]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# 当前span在文本中的起始标记索引。
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# 当前span在文本中的结束标记索引。
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# 检测答案是否超出span范围（如果超出范围，该特征将以CLS标记索引标记）。
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # 将token_start_index和token_end_index移动到答案的两端。
    # 注意：如果答案是最后一个单词，我们可以移到最后一个标记之后（边界情况）。
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("答案不在此特征中。")

18 19


打印检查是否准确找到了起始位置：

In [25]:
# 通过查找 offset mapping 位置，解码 context 中的答案 
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
# 直接打印 数据集中的标准答案（answer["text"])
print(answers["text"][0])

jay z
Jay Z


#### 关于填充的策略

- 对于没有超过最大长度的文本，填充补齐长度。
- 对于需要左侧填充的模型，交换 question 和 context 顺序

In [26]:
pad_on_right = tokenizer.padding_side == "right"

### 整合以上所有预处理步骤

让我们将所有内容整合到一个函数中，并将其应用到训练集。

针对不可回答的情况（上下文过长，答案在另一个特征中），我们为开始和结束位置都设置了cls索引。

如果allow_impossible_answers标志为False，我们还可以简单地从训练集中丢弃这些示例。

In [27]:
def prepare_train_features(examples):
    # 一些问题的左侧可能有很多空白字符，这对我们没有用，而且会导致上下文的截断失败
    # （标记化的问题将占用大量空间）。因此，我们删除左侧的空白字符。
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # 使用截断和填充对我们的示例进行标记化，但保留溢出部分，使用步幅（stride）。
    # 当上下文很长时，这会导致一个示例可能提供多个特征，其中每个特征的上下文都与前一个特征的上下文有一些重叠。
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # 由于一个示例可能给我们提供多个特征（如果它具有很长的上下文），我们需要一个从特征到其对应示例的映射。这个键就提供了这个映射关系。
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # 偏移映射将为我们提供从令牌到原始上下文中的字符位置的映射。这将帮助我们计算开始位置和结束位置。
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # 让我们为这些示例进行标记！
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # 我们将使用 CLS 特殊 token 的索引来标记不可能的答案。
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # 获取与该示例对应的序列（以了解上下文和问题是什么）。
        sequence_ids = tokenized_examples.sequence_ids(i)

        # 一个示例可以提供多个跨度，这是包含此文本跨度的示例的索引。
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # 如果没有给出答案，则将cls_index设置为答案。
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # 答案在文本中的开始和结束字符索引。
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # 当前跨度在文本中的开始令牌索引。
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # 当前跨度在文本中的结束令牌索引。
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # 检测答案是否超出跨度（在这种情况下，该特征的标签将使用CLS索引）。
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # 否则，将token_start_index和token_end_index移到答案的两端。
                # 注意：如果答案是最后一个单词（边缘情况），我们可以在最后一个偏移之后继续。
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

#### datasets.map 的进阶使用

使用 `datasets.map` 方法将 `prepare_train_features` 应用于所有训练、验证和测试数据：

- batched: 批量处理数据。
- remove_columns: 因为预处理更改了样本的数量，所以在应用它时需要删除旧列。
- load_from_cache_file：是否使用datasets库的自动缓存

datasets 库针对大规模数据，实现了高效缓存机制，能够自动检测传递给 map 的函数是否已更改（因此需要不使用缓存数据）。如果在调用 map 时设置 `load_from_cache_file=False`，可以强制重新应用预处理。

In [28]:
tokenized_datasets = datasets.map(prepare_train_features,
                                  batched=True,
                                  remove_columns=datasets["train"].column_names)

Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

## 微调模型

现在我们的数据已经准备好用于训练，我们可以下载预训练模型并进行微调。

由于我们的任务是问答，我们使用 `AutoModelForQuestionAnswering` 类。(对比 Yelp 评论打分使用的是 `AutoModelForSequenceClassification` 类）

警告通知我们正在丢弃一些权重（`vocab_transform` 和 `vocab_layer_norm` 层），并随机初始化其他一些权重（`pre_classifier` 和 `classifier` 层）。在微调模型情况下是绝对正常的，因为我们正在删除用于预训练模型的掩码语言建模任务的头部，并用一个新的头部替换它，对于这个新头部，我们没有预训练的权重，所以库会警告我们在用它进行推理之前应该对这个模型进行微调，而这正是我们要做的事情。

In [29]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at /data/models/distilbert--distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### 训练超参数（TrainingArguments）

In [30]:
batch_size=64
model_dir = f"/data/models/distilbert--distilbert-base-uncased-finetuned-squad"

args = TrainingArguments(
    output_dir=model_dir,
    eval_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

#### Data Collator（数据整理器）

数据整理器将训练数据整理为批次数据，用于模型训练时的批次处理。本教程使用默认的 `default_data_collator`。

In [31]:
from transformers import default_data_collator

data_collator = default_data_collator

### 实例化训练器（Trainer）

为了减少训练时间（需要大量算力支持），我们不在本教程的训练模型过程中计算模型评估指标。

而是训练完成后，再独立进行模型评估。

In [32]:
# 当前没有算力，仅笔记本CPU来运行，先缩小训练集合大小，了解整个微调过程。
small_train_dataset = tokenized_datasets["train"].shuffle(seed=66).select(range(100))
small_eval_dataset = tokenized_datasets["validation"].shuffle(seed=66).select(range(100))

In [33]:
trainer = Trainer(
    model,
    args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

C:\Users\ThinkPad\AppData\Local\Temp\ipykernel_19252\4021240430.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [34]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,5.990912
2,No log,5.942616
3,No log,5.922647


TrainOutput(global_step=6, training_loss=5.942782084147136, metrics={'train_runtime': 763.8786, 'train_samples_per_second': 0.393, 'train_steps_per_second': 0.008, 'total_flos': 29396947507200.0, 'train_loss': 5.942782084147136, 'epoch': 3.0})

In [35]:
model_to_save = trainer.save_model(model_dir)

## 模型评估

**评估模型输出需要一些额外的处理：将模型的预测映射回上下文的部分。**

模型直接输出的是预测答案的`起始位置`和`结束位置`的**logits**

In [36]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

模型的输出是一个类似字典的对象，其中包含损失（因为我们提供了标签），以及起始和结束logits。我们不需要损失来进行预测，让我们看一下logits：

In [37]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([64, 384]), torch.Size([64, 384]))

In [38]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([121,  64,  47, 142,  63,  72,  94, 116,  24,  38, 134, 125,  99,  14,
         156,  27, 120,  45,  38,  72, 129,  27,  43,  22,  10,  68, 126, 107,
         121,  98,  29, 160, 210, 138,  22,  69, 331,  19,  16,  39, 101, 233,
          75, 101,  60,  50,  46,  68,  28,  83,  71, 106, 155,  64, 150,   4,
          87, 105,  37, 163,  18,  84, 144,   3]),
 tensor([  0, 177, 105, 120,  77,  24, 266, 120, 125, 162, 106, 113,  35, 133,
         132,  51,  17, 125, 136, 175,   0, 262, 110, 115,  72, 153, 127, 134,
         112,   0,  78, 110, 126,  36,  45, 290, 127, 159, 130, 109, 326,  31,
          12,  22,   4, 109, 138, 130, 161, 126, 269,  37,  97, 132, 128, 140,
          96, 152, 167,  27, 132, 101, 116, 117]))

#### 如何从模型输出的位置 logit 组合成答案

我们有每个特征和每个标记的logit。在每个特征中为每个标记预测答案最明显的方法是，将起始logits的最大索引作为起始位置，将结束logits的最大索引作为结束位置。

在许多情况下这种方式效果很好，但是如果此预测给出了不可能的结果该怎么办？比如：起始位置可能大于结束位置，或者指向问题中的文本片段而不是答案。在这种情况下，我们可能希望查看第二好的预测，看它是否给出了一个可能的答案，并选择它。

选择第二好的答案并不像选择最佳答案那么容易：
- 它是起始logits中第二佳索引与结束logits中最佳索引吗？
- 还是起始logits中最佳索引与结束logits中第二佳索引？
- 如果第二好的答案也不可能，那么对于第三好的答案，情况会更加棘手。

为了对答案进行分类，
1. 将使用通过添加起始和结束logits获得的分数
1. 设计一个名为`n_best_size`的超参数，限制不对所有可能的答案进行排序。
1. 我们将选择起始和结束logits中的最佳索引，并收集这些预测的所有答案。
1. 在检查每一个是否有效后，我们将按照其分数对它们进行排序，并保留最佳的答案。

以下是我们如何在批次中的第一个特征上执行此操作的示例：

In [39]:
n_best_size = 20

In [40]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()

# 获取最佳的起始和结束位置的索引：
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()

valid_answers = []

# 遍历起始位置和结束位置的索引组合
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index:  # 需要进一步测试以检查答案是否在上下文中
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": ""  # 我们需要找到一种方法来获取与上下文中答案对应的原始子字符串
                }
            )


然后，我们可以根据它们的得分对`valid_answers`进行排序，并仅保留最佳答案。唯一剩下的问题是如何检查给定的跨度是否在上下文中（而不是问题中），以及如何获取其中的文本。为此，我们需要向我们的验证特征添加两个内容：

- 生成该特征的示例的ID（因为每个示例可以生成多个特征，如前所示）；
- 偏移映射，它将为我们提供从标记索引到上下文中字符位置的映射。

这就是为什么我们将使用以下函数稍微不同于`prepare_train_features`来重新处理验证集：

In [41]:
def prepare_validation_features(examples):
    # 一些问题的左侧有很多空白，这些空白并不有用且会导致上下文截断失败（分词后的问题会占用很多空间）。
    # 因此我们移除这些左侧空白
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # 使用截断和可能的填充对我们的示例进行分词，但使用步长保留溢出的令牌。这导致一个长上下文的示例可能产生
    # 几个特征，每个特征的上下文都会稍微与前一个特征的上下文重叠。
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # 由于一个示例在上下文很长时可能会产生几个特征，我们需要一个从特征映射到其对应示例的映射。这个键就是为了这个目的。
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # 我们保留产生这个特征的示例ID，并且会存储偏移映射。
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # 获取与该示例对应的序列（以了解哪些是上下文，哪些是问题）。
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # 一个示例可以产生几个文本段，这里是包含该文本段的示例的索引。
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # 将不属于上下文的偏移映射设置为None，以便容易确定一个令牌位置是否属于上下文。
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples


将`prepare_validation_features`应用到验证集： 限于算力，随机取200个样本

In [48]:
small_validation_dataset_200 = datasets["validation"].shuffle(seed=66).select(range(200))

In [49]:
validation_features = small_validation_dataset_200.map(
    prepare_validation_features,
    batched=True,
    remove_columns=small_validation_dataset_200.column_names
)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Now we can grab the predictions for all features by using the `Trainer.predict` method:

In [50]:
raw_predictions = trainer.predict(validation_features)

`Trainer`会隐藏模型不使用的列（在这里是`example_id`和`offset_mapping`，我们需要它们进行后处理），所以我们需要将它们重新设置回来：

In [51]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

现在，我们可以改进之前的测试：

由于在偏移映射中，当它对应于问题的一部分时，我们将其设置为None，因此可以轻松检查答案是否完全在上下文中。我们还可以从考虑中排除非常长的答案（可以调整的超参数）。

展开说下具体实现：
- 首先从模型输出中获取起始和结束的逻辑值（logits），这些值表明答案在文本中可能开始和结束的位置。
- 然后，它使用偏移映射（offset_mapping）来找到这些逻辑值在原始文本中的具体位置。
- 接下来，代码遍历可能的开始和结束索引组合，排除那些不在上下文范围内或长度不合适的答案。
- 对于有效的答案，它计算出一个分数（基于开始和结束逻辑值的和），并将答案及其分数存储起来。
- 最后，它根据分数对答案进行排序，并返回得分最高的几个答案。

In [53]:
max_answer_length = 30

In [67]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]

# 第一个特征来自第一个示例。对于更一般的情况，我们需要将example_id匹配到一个示例索引
context = small_validation_dataset_200[4]["context"]

# 收集最佳开始/结束逻辑的索引：
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # 不考虑超出范围的答案，原因是索引超出范围或对应于输入ID的部分不在上下文中。
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # 不考虑长度小于0或大于max_answer_length的答案。
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # 我们需要细化这个测试，以检查答案是否在上下文中
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers


[{'score': 0.47923446,
  'text': ' the left banks of the Upper Rhine was sold to Burgundy by Archduke Sigismund o'},
 {'score': 0.43272084,
  'text': 'banks of the Upper Rhine was sold to Burgundy by Archduke Sigismund o'},
 {'score': 0.4242437,
  'text': 'of the Upper Rhine was sold to Burgundy by Archduke Sigismund o'},
 {'score': 0.4019267,
  'text': ' the left banks of the Upper Rhine was sold to Burgundy by Archduke Sigismund of Austria in 1469 and '},
 {'score': 0.37975088,
  'text': ' the left banks of the Upper Rhine was sold to Burgundy by Archduke'},
 {'score': 0.35541308,
  'text': 'banks of the Upper Rhine was sold to Burgundy by Archduke Sigismund of Austria in 1469 and '},
 {'score': 0.34693593,
  'text': 'of the Upper Rhine was sold to Burgundy by Archduke Sigismund of Austria in 1469 and '},
 {'score': 0.3404683,
  'text': ' the left banks of the Upper Rhine was sold to Bu'},
 {'score': 0.3332373,
  'text': 'banks of the Upper Rhine was sold to Burgundy by Archduke'},
 

打印比较模型输出和标准答案（Ground-truth）是否一致:

In [68]:
small_validation_dataset_200[4]["answers"]

{'text': ['Archduke Sigismund',
  'Archduke Sigismund of Austria',
  'Archduke Sigismund of Austria'],
 'answer_start': [659, 659, 659]}

**模型最高概率的输出与标准答案一致**

正如上面的代码所示，这在第一个特征上很容易，因为我们知道它来自第一个示例。

对于其他特征，我们需要建立一个示例与其对应特征的映射关系。

此外，由于一个示例可以生成多个特征，我们需要将由给定示例生成的所有特征中的所有答案汇集在一起，然后选择最佳答案。

下面的代码构建了一个示例索引到其对应特征索引的映射关系：

In [69]:
import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

当`squad_v2 = True`时，有一定概率出现不可能的答案（impossible answer)。

上面的代码仅保留在上下文中的答案，我们还需要获取不可能答案的分数（其起始和结束索引对应于CLS标记的索引）。

当一个示例生成多个特征时，我们必须在所有特征中的不可能答案都预测出现不可能答案时（因为一个特征可能之所以能够预测出不可能答案，是因为答案不在它可以访问的上下文部分），这就是为什么一个示例中不可能答案的分数是该示例生成的每个特征中的不可能答案的分数的最小值。

In [70]:
squad_v2 = True

In [71]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # 构建一个从示例到其对应特征的映射。
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # 我们需要填充的字典。
    predictions = collections.OrderedDict()

    # 日志记录。
    print(f"正在后处理 {len(examples)} 个示例的预测，这些预测分散在 {len(features)} 个特征中。")

    # 遍历所有示例！
    for example_index, example in enumerate(tqdm(examples)):
        # 这些是与当前示例关联的特征的索引。
        feature_indices = features_per_example[example_index]

        min_null_score = None # 仅在squad_v2为True时使用。
        valid_answers = []
        
        context = example["context"]
        # 遍历与当前示例关联的所有特征。
        for feature_index in feature_indices:
            # 我们获取模型对这个特征的预测。
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # 这将允许我们将logits中的某些位置映射到原始上下文中的文本跨度。
            offset_mapping = features[feature_index]["offset_mapping"]

            # 更新最小空预测。
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # 浏览所有的最佳开始和结束logits，为 `n_best_size` 个最佳选择。
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # 不考虑超出范围的答案，原因是索引超出范围或对应于输入ID的部分不在上下文中。
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # 不考虑长度小于0或大于max_answer_length的答案。
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # 在极少数情况下我们没有一个非空预测，我们创建一个假预测以避免失败。
            best_answer = {"text": "", "score": 0.0}
        
        # 选择我们的最终答案：最佳答案或空答案（仅适用于squad_v2）
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions


在原始结果上应用后处理问答结果：

In [73]:
final_predictions = postprocess_qa_predictions(small_validation_dataset_200, validation_features, raw_predictions.predictions)

正在后处理 200 个示例的预测，这些预测分散在 204 个特征中。


  0%|          | 0/200 [00:00<?, ?it/s]

加载 `SQuAD v2` 的评估指标

In [74]:
#from datasets import load_metric
#metric = load_metric("squad_v2" if squad_v2 else "squad")

In [76]:
import evaluate
metric = evaluate.load("/data/models/evaluate-0.4.5/metrics/squad_v2" if squad_v2 else "/data/models/evaluate-0.4.5/metrics/squad")

接下来，我们可以调用上面定义的函数进行评估。

只需稍微调整一下预测和标签的格式，因为它期望的是一系列字典而不是一个大字典。

在使用`squad_v2`数据集时，我们还需要设置`no_answer_probability`参数（我们在这里将其设置为0.0，因为如果我们选择了答案，我们已经将答案设置为空）。

In [78]:
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in small_validation_dataset_200]
metric.compute(predictions=formatted_predictions, references=references)

{'exact': 1.0,
 'f1': 4.616015014636297,
 'total': 200,
 'HasAns_exact': 0.0,
 'HasAns_f1': 7.69364896731127,
 'HasAns_total': 94,
 'NoAns_exact': 1.8867924528301887,
 'NoAns_f1': 1.8867924528301887,
 'NoAns_total': 106,
 'best_exact': 53.0,
 'best_exact_thresh': 0.0,
 'best_f1': 53.07952551834131,
 'best_f1_thresh': 0.0}

### Homework：加载本地保存的模型，进行评估和再训练更高的 F1 Score

#### **换更大一些的训练集**

In [96]:
trained_model = AutoModelForQuestionAnswering.from_pretrained(model_dir)

In [97]:
small_train_dataset_800 = tokenized_datasets["train"].shuffle(seed=66).select(range(800))
small_eval_dataset_800 = tokenized_datasets["validation"].shuffle(seed=66).select(range(800))

In [98]:
trained_trainer = Trainer(
    trained_model,
    args,
    train_dataset=small_train_dataset_800,
    eval_dataset=small_eval_dataset_800,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

C:\Users\ThinkPad\AppData\Local\Temp\ipykernel_19252\2344870709.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trained_trainer = Trainer(


In [99]:
trained_trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,5.235181
2,No log,4.458299
3,No log,4.115832


TrainOutput(global_step=39, training_loss=5.003631983047876, metrics={'train_runtime': 8894.3812, 'train_samples_per_second': 0.27, 'train_steps_per_second': 0.004, 'total_flos': 235175580057600.0, 'train_loss': 5.003631983047876, 'epoch': 3.0})

In [101]:
model_dir2 = f"/data/models/distilbert--distilbert-base-uncased-finetuned-squad-800"

In [102]:
model_to_save = trained_trainer.save_model(model_dir2)

In [103]:
raw_predictions = trained_trainer.predict(validation_features)

In [104]:
final_predictions = postprocess_qa_predictions(small_validation_dataset_200, validation_features, raw_predictions.predictions)

正在后处理 200 个示例的预测，这些预测分散在 204 个特征中。


  0%|          | 0/200 [00:00<?, ?it/s]

In [105]:
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in small_validation_dataset_200]
metric.compute(predictions=formatted_predictions, references=references)

{'exact': 53.0,
 'f1': 53.0,
 'total': 200,
 'HasAns_exact': 0.0,
 'HasAns_f1': 0.0,
 'HasAns_total': 94,
 'NoAns_exact': 100.0,
 'NoAns_f1': 100.0,
 'NoAns_total': 106,
 'best_exact': 53.0,
 'best_exact_thresh': 0.0,
 'best_f1': 53.0,
 'best_f1_thresh': 0.0}

*以上，更大的训练集可以显著提升F1 Score*

#### **在小规模训练集上，增加epochs的次数**

因使用相同的训练集，看起来F1 Score是一样的，但是Training Loss和Validation Loss明显减小了。

In [106]:
args800 = TrainingArguments(
    output_dir=model_dir2,
    eval_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=30,
    save_total_limit=5,
    num_train_epochs=5,
    weight_decay=0.01,
)

In [107]:
trained_model_800 = AutoModelForQuestionAnswering.from_pretrained(model_dir2)

In [109]:
trained_trainer_800 = Trainer(
    trained_model_800,
    args800,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

C:\Users\ThinkPad\AppData\Local\Temp\ipykernel_19252\4005132131.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trained_trainer_800 = Trainer(


In [110]:
trained_trainer_800.train()

Epoch,Training Loss,Validation Loss
1,No log,2.755088
2,3.506300,2.698053
3,3.506300,2.680476
4,2.886900,2.751134
5,2.489000,2.756043


TrainOutput(global_step=95, training_loss=2.938158336438631, metrics={'train_runtime': 3540.9303, 'train_samples_per_second': 0.424, 'train_steps_per_second': 0.027, 'total_flos': 146984737536000.0, 'train_loss': 2.938158336438631, 'epoch': 5.0})

In [111]:
model_dir3 = f"/data/models/distilbert--distilbert-base-uncased-finetuned-squad-800-100"
model_to_save_3 = trained_trainer_800.save_model(model_dir3)

In [112]:
raw_predictions_800 = trained_trainer_800.predict(validation_features)

In [116]:
final_predictions_800 = postprocess_qa_predictions(small_validation_dataset_200, validation_features, raw_predictions_800.predictions)

正在后处理 200 个示例的预测，这些预测分散在 204 个特征中。


  0%|          | 0/200 [00:00<?, ?it/s]

In [115]:
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions_800.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions_800.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in small_validation_dataset_200]
metric.compute(predictions=formatted_predictions, references=references)

{'exact': 53.0,
 'f1': 53.0,
 'total': 200,
 'HasAns_exact': 0.0,
 'HasAns_f1': 0.0,
 'HasAns_total': 94,
 'NoAns_exact': 100.0,
 'NoAns_f1': 100.0,
 'NoAns_total': 106,
 'best_exact': 53.0,
 'best_exact_thresh': 0.0,
 'best_f1': 53.0,
 'best_f1_thresh': 0.0}